In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
# POPULATE ROOM RESERVATION
# gather data
# - get rooms and dates
room_names=pd.read_excel('populate_transactional.xlsx',sheet_name='Room',header=None,squeeze=True).str.split(',',expand=True)
room_names,date_names=room_names.iloc[:,0],room_names.iloc[:,1]
room_names=room_names.str.split(' \'',expand=True).iloc[:,1].str.slice(0,-1)
exec_phrase='EXEC INSERT_ROOM_RESERVATION'
# - get people and cards
people_names=pd.read_excel('populate_transactional.xlsx',sheet_name='People',header=None,squeeze=True)
# - get room to roomtype
room2roomtype=pd.read_excel('populate_transactional.xlsx',sheet_name='Room2RoomType',header=None,squeeze=True)
room2roomtype=room2roomtype.str.split(',',expand=True).set_index(0)[1].to_dict()
roomtype2count={'Cardio Room':range(0,4),
                'Sports Room':range(0,16),
                'Yoga Room':range(0,10),
                'Weight Room':range(0,3),
                'Locker Room':range(0,1),
                'Sauna':range(1,11),
                'Pool':range(0,8)}
# - get dates
dates=pd.read_excel('populate_transactional.xlsx',sheet_name='Date',header=None,squeeze=True)
# - get feeplans
feeplans=['LargeDeposit']*62+['MediumDeposit']*26+['SmallDeposit']*12  # try to create population percentages
# create fake entries
np.random.seed(0)
num_people=people_names.shape[0]//4//3*3  # num people per room = 25%, rounded to nearest multiple of 3
for room in room_names:
    count=np.random.choice(roomtype2count[room2roomtype[room]],size=1)[0]
    choices=[count]
    steps=num_people  # default
    if(count-1>0):
        choices=list(range(count-1,count+2))
        steps=num_people//3  # num people assigned to each step
    peoples_idx=np.random.choice(people_names.index,size=num_people,replace=False)
    idx=0
    for choice in choices:  # choices=list of counts, steps=# of people
        for _ in range(steps):
            people_idx=peoples_idx[idx]
            feeplan='PerHour'
            if(people_names.iloc[people_idx,0]=='Credit'):
                feeplan=np.random.choice(feeplans,size=1,replace=False)[0]
            people_name=people_names.iloc[people_idx,1]
            for _ in range(choice):
                startdate,enddate=np.random.choice(dates,size=2,replace=False)
                startdate,enddate=datetime.datetime.strptime(startdate[:-1],'%Y/%m/%d'),datetime.datetime.strptime(enddate[:-1],'%Y/%m/%d')
                start,end=str(min(startdate,enddate).date()),str(max(startdate,enddate).date())
                print(exec_phrase+' \''+'\',\''.join([room,people_name.strip('\''),feeplan])+'\',\''+'\',\''.join([start,end])+'\'')
            idx+=1  # take a step

In [ ]:
# POPULATE ROOM RESERVATION
# gather data
# - get reservation data
reservation_names=pd.read_excel('populate_transactional.xlsx',sheet_name='RoomReservation')
columns=reservation_names.columns
# - get room to roomtype
room2roomtype=pd.read_excel('populate_transactional.xlsx',sheet_name='Room2RoomType',header=None,squeeze=True)
room2roomtype=room2roomtype.str.split(',',expand=True).set_index(0)[1].to_dict()
reservation_names['RoomType']=reservation_names['Roomname'].map(room2roomtype)
# - get class to classtype -- not necessary but more info
class_names=pd.read_excel('populate_transactional.xlsx',sheet_name='ClassAndClassType',header=None,squeeze=True).str.split(',',expand=True)
class2classtype=class_names.set_index(0)[1].to_dict()
# define roomtype to class
roomtype2class={'Yoga Room':['Beginner Yoga','Intermediate Yoga','Advanced Yoga','Hot Yoga'],
                'Weight Room':['Bodyweight HIIT','Strength Circuit'],
                'Pool':['Aqua Yoga'],
                'Sports Room':['Sport-Based Circuit','General Kickboxing','Self-Defense Kickboxing'],
                'Cardio Room':['Cardio Circuit','Aerobic-Spin HIIT','Aerobic-Track HIIT']}
# define class distributions - as in class,num where num=percent of rows assigned to a class
class2num={'Beginner Yoga':0.1,'Intermediate Yoga':0.05,'Advanced Yoga':0.025,'Hot Yoga':0.025,
           'Bodyweight HIIT':0.05,'Strength Circuit':0.1,'Aqua Yoga':0.01,
           'Sport-Based Circuit':0.05,'General Kickboxing':0.025,'Self-Defense Kickboxing':0.01,
           'Cardio Circuit':0.05,'Aerobic-Spin HIIT':0.05,'Aerobic-Track HIIT':0.05}
# grab rows
import math
np.random.seed(0)
for roomtype in roomtype2class:
    # gather data
    classes=roomtype2class[roomtype]
    reservation_names_subset=reservation_names.loc[reservation_names['RoomType']==roomtype,columns]
    nrows=reservation_names_subset.shape[0]
    # get pickings
    classes_nums=[math.ceil(class2num[class_name]*nrows) for class_name in classes]
    chosen_idxs=np.random.choice(reservation_names_subset.index,size=sum(classes_nums),replace=False)
    reservation_names_subset=reservation_names_subset.loc[chosen_idxs]
    # assign classes
    class_names=[]
    for idx,class_name in enumerate(classes):
        class_names.extend(classes_nums[idx]*[class_name])
    reservation_names_subset['ClassName']=class_names
    # print results
    rows=('EXEC INSERT_CLASS_ROOM_RESERVATION \''+reservation_names_subset.agg('\',\''.join,axis=1)+'\'')
    for row in rows:
        print(row)

In [ ]:
# POPULATE EQUIPMENT
# - get room to roomtype
room2roomtype=pd.read_excel('populate_transactional.xlsx',sheet_name='Room2RoomType',header=None,squeeze=True)
room2roomtype=room2roomtype.str.split(',',expand=True).set_index(0)[1].to_dict()
# - define source and materials
# - [roomname,[roomsubtypes,[[equipmenttype,[equipmentname,equipmentquantity]]]]]
equipment={'Cardio Room':('CR',
            {'A':[[['Cardio','C'],[['Elliptical',6],['Treadmill',10],['Exercise Bike',6],['Stair Stepper',2]]]],
             'B':[[['Cardio','C'],[['Track',1],['Treadmill',2]]]]}),
           'Weight Room':('WR',
            {'A':[[['Weight-Freehand','WF'],[['Dumbbell-10lb',20],['Dumbbell-20lb',20],['Dumbbell-30lb',20],
                                      ['Dumbbell-40lb',20],['Dumbbell-50lb',20],['Dumbbell-60lb',20],
                                      ['Medicine-ball-10lb',10],['Medicine-ball-20lb',10],['Medicine-ball-30lb',10],
                                      ['Medicine-ball-40lb',10],['Medicine-ball-50lb',10],['Medicine-ball-60lb',10]]],
                  [['Bench Press','BP'],[['Base-bar-35lb',5],['Base-bar-45lb',3],['Weight-5lb',10],
                                         ['Weight-10lb',10],['Weight-15lb',10],['Weight-20lb',10],
                                         ['Weight-25lb',10],['Weight-30lb',10],['Weight-35lb',10]]]],
             'B':[[['Weight-Freehand','WF'],[['Dumbbell-10lb',20],['Dumbbell-20lb',20],['Dumbbell-30lb',20],
                                      ['Dumbbell-40lb',20],['Dumbbell-50lb',20],['Dumbbell-60lb',20],
                                      ['Medicine-ball-10lb',10],['Medicine-ball-20lb',10],['Medicine-ball-30lb',10],
                                      ['Medicine-ball-40lb',10],['Medicine-ball-50lb',10],['Medicine-ball-60lb',10]]],
                  [['Bench Press','BP'],[['Base-bar-35lb',5],['Base-bar-45lb',3],['Weight-5lb',10],
                                         ['Weight-10lb',10],['Weight-15lb',10],['Weight-20lb',10],
                                         ['Weight-25lb',10],['Weight-30lb',10],['Weight-35lb',10]]],
                  [['Weight-Machine','WM'],[['Leg-press',2],['Arm-press',2],['Back-press',2]]]]}),
           'Pool':('PL',
            {'A':[[['Pool','P'],[['Kickboard',5],['Flippers',3],['Pull-buoy',2]]]],
             'B':[[['Pool','P'],[['Flippers',2],['Snorkel',2],['Dive-equipment',1]]]],
             'C':[[['Pool','P'],[['Kickboard',10],['Flippers',5],['Pull-buoy',3]]]]}),
          'Yoga Room':('YR',
           {'A':[[['Stretching','ST'],[['Yoga-mat',2],['Foam-cushion',1]]]],
            'B':[[['Stretching','ST'],[['Yoga-mat',10],['Foam-cushion',5],['Yoga-block',5]]]]}),
          'Sport Room':('SR',
           {'A':[[['Sport','SP'],[['Basketball',10],['Volleyball',10],['Badminton-racket',10],['Badminton',10]]]],
            'B':[[['Sport','SP'],[['Racquetball-racket',4],['Racuqetball-ball',6],['Squash-racket',2],['Squash-ball',4]]]],
            'C':[[['Sport','SP'],[['Chalk-5lb',10],['Foot-gear',20],['Harness',10]]]]})}
# - reconfigure room-->roomtype
roomtype2room={}
for room,roomtype in room2roomtype.items():
    if(roomtype not in ['Locker Room','Sauna']):
        if(roomtype not in roomtype2room):
            roomtype2room[roomtype]=[]
        roomtype2room[roomtype].append(room)
# - process
for roomtype,rooms in roomtype2room.items():
    for room in rooms:
        roomtype,subtype=room.split('-')
        suffix=subtype
        subtype=subtype[1]
        # grab data
        data=equipment[roomtype]
        prefix,data=data[0],data[1]
        data=data[subtype]
        for prefix_info,data_info in data:
            equip_type,equip_prefix=prefix_info
            for equip_name,equip_num in data_info:
                for equip_idx in range(1,equip_num+1):
                    name=suffix+':'+prefix+'-'+equip_prefix+':'+equip_name+'-'+str(equip_idx)
                    print('EXEC INSERT_EQUIPMENT \''+'\', \''.join([equip_type,room,name])+'\', NULL')

In [ ]:
# POPULATE CHECK -- produces some (~400 of ~20k rows) of un-enterable data bc of the debit, pool, checkout rule
# - get equipment to equipment type
e2et_quants=[('Pool',range(2,15)),('Cardio',range(35,150)),('Weight-Freehand',range(50,200)),
             ('Bench Press',range(40,180)),('Stretching',range(5,25)),('Sport',range(25,100)),
             ('Weight-Machine',range(30,150))]
e2et=pd.read_excel('populate_transactional.xlsx',sheet_name='Equipment2EquipmentType')
# - get people
peoples=pd.read_excel('populate_transactional.xlsx',sheet_name='People',header=None).iloc[:,1]
# - get dates
dates=pd.read_excel('populate_transactional.xlsx',sheet_name='Date',header=None,squeeze=True)
# - get checks
checks=['In','Out']
# - get conditions
conditions=[['New',0.15],['Good',0.35],['Worn',0.45],['Damaged',0.05]]
# - get rows
import math
np.random.seed(0)
counter=0
for people in peoples:
    for check in checks:
        for equipment_type,count_choices in e2et_quants:
            count=np.random.choice(list(count_choices),size=1,replace=False)[0]
            # get condition
            condition_count=math.ceil(count/100)*200
            condition_pool=[]
            for condition,perc in conditions:
                if(check!='Out' or condition!='Damaged'):
                    condition_pool.extend([condition]*int(perc*condition_count))
            condition_pool=np.random.choice(condition_pool,size=count,replace=False)
            equipment_pool=(e2et['EquipmentName'][e2et['EquipmentTypeName']==equipment_type]).values
            equipment_pool=np.random.choice(equipment_pool,size=count,replace=True)
            date_pool=np.random.choice(dates,size=count,replace=False)
            for idx,condition in enumerate(condition_pool):
                equipment_name=equipment_pool[idx]
                date_name=date_pool[idx]
                hour,minutes,seconds,milliseconds=np.random.randint(0,24),np.random.randint(0,60),np.random.randint(0,60),np.random.randint(0,1000)
                time=str(hour).zfill(2)+':'+str(minutes).zfill(2)+':'+str(seconds).zfill(2)+'.'+str(milliseconds).zfill(3)
                results=[equipment_name,check,condition,date_name[:-1]+' '+time+'\'']
#                 print('EXEC INSERT_CHECK '+people+',\''+'\',\''.join(results))
                counter+=1